# Run benchmarking experiments
Note: The Benchmarking Framework does currently not properly support auto-regression or lagged covariates with multiple step ahead forecasts.

In [1]:
import os
import pandas as pd
from neuralprophet import NeuralProphet, set_log_level

set_log_level("ERROR")

## Load data

In [2]:
%%capture
# create temporary data folder and download data
!rm -r tmp-data
!mkdir tmp-data
!wget -P tmp-data https://raw.githubusercontent.com/ourownstory/neuralprophet-data/main/datasets/air_passengers.csv
# !wget -P tmp-data https://raw.githubusercontent.com/ourownstory/neuralprophet-data/main/datasets/wp_log_peyton_manning.csv
# !wget -P tmp-data https://raw.githubusercontent.com/ourownstory/neuralprophet-data/main/datasets/retail_sales.csv
# !wget -P tmp-data https://raw.githubusercontent.com/ourownstory/neuralprophet-data/main/datasets/yosemite_temps.csv
# !wget -P tmp-data https://raw.githubusercontent.com/ourownstory/neuralprophet-data/main/datasets/energy/ERCOT_load_2004_2021Sept.csv

In [3]:
os.listdir('tmp-data')

['air_passengers.csv']

In [4]:
air_passengers_df = pd.read_csv(os.path.join('tmp-data', 'air_passengers.csv'))
# peyton_manning_df = pd.read_csv(os.path.join('tmp-data', 'wp_log_peyton_manning.csv'))
# retail_sales_df = pd.read_csv(os.path.join('tmp-data', 'retail_sales.csv'))
# yosemite_temps_df = pd.read_csv(os.path.join('tmp-data', 'yosemite_temps.csv'))
# ercot_load_df = pd.read_csv(os.path.join('tmp-data', 'ERCOT_load_2004_2021Sept.csv'))[['ds', 'y']]

# Streamlined 

In [5]:
from neuralprophet.benchmark import Dataset, NeuralProphetModel, SimpleExperiment, CrossValidationExperiment
from neuralprophet.benchmark import SimpleBenchmark, CrossValidationBenchmark

## Streamlined Simple

In [6]:
ts = Dataset(df = air_passengers_df, name = "air_passengers", freq = "MS")
params = {"seasonality_mode": "multiplicative"}
exp = SimpleExperiment(
    model_class=NeuralProphetModel,
    params=params,
    data=ts,
    metrics=["MAE", "MSE"],
    test_percentage=25,
)
result_train, result_val = exp.fit()

  0%|          | 0/203 [00:00<?, ?it/s]

In [7]:
result_val

{'data': 'air_passengers',
 'model': 'NeuralProphet',
 'params': "{'seasonality_mode': 'multiplicative'}",
 'MAE': 24.757139205932617,
 'MSE': 837.4072265625}

## Streamlined CV


In [8]:
ts = Dataset(df = air_passengers_df, name = "air_passengers", freq = "MS")
params = {"seasonality_mode": "multiplicative"}
exp_cv = CrossValidationExperiment(
    model_class=NeuralProphetModel,
    params=params,
    data=ts,
    metrics=["MAE", "MSE"],
    test_percentage=10,
    num_folds=3,
    fold_overlap_pct=0,
  )
result_train, result_val = exp_cv.fit()

  0%|          | 0/202 [00:00<?, ?it/s]

  0%|          | 0/205 [00:00<?, ?it/s]

  0%|          | 0/207 [00:00<?, ?it/s]

In [9]:
result_val

{'data': 'air_passengers',
 'model': 'NeuralProphet',
 'params': "{'seasonality_mode': 'multiplicative'}",
 'MAE': [16.760211944580078, 12.247063636779785, 28.325775146484375],
 'MSE': [443.76409912109375, 248.8365020751953, 860.5340576171875]}

# Streamlined Benchmark

In [10]:
dataset_list = [
    Dataset(df = air_passengers_df, name = "air_passengers", freq = "MS"),
    # Dataset(df = peyton_manning_df, name = "peyton_manning", freq = "D"),
    # Dataset(df = retail_sales_df, name = "retail_sales", freq = "D"),
    # Dataset(df = yosemite_temps_df, name = "yosemite_temps", freq = "5min"),
    # Dataset(df = ercot_load_df, name = "ercot_load", freq = "H"),
]
model_classes_and_params = [
    (NeuralProphetModel, {}),
    (NeuralProphetModel, {"n_changepoints": 5}),
    (NeuralProphetModel, {"seasonality_mode": "multiplicative", "learning_rate": 0.1}),
]

In [11]:
benchmark = SimpleBenchmark(
    model_classes_and_params=model_classes_and_params, # iterate over this list of tuples
    datasets=dataset_list, # iterate over this list
    metrics=["MAE", "MSE"],
    test_percentage=25,
)
results_train, results_val = benchmark.run()


  0%|          | 0/203 [00:00<?, ?it/s]

  0%|          | 0/203 [00:00<?, ?it/s]

In [12]:
results_val

,data,model,params,MAE,MSE
0,air_passengers,NeuralProphet,{},38.343773,1896.185059
1,air_passengers,NeuralProphet,{'n_changepoints': 5},39.860699,2000.540527
2,air_passengers,NeuralProphet,"{'seasonality_mode': 'multiplicative', 'learni...",24.721510,839.445984


In [13]:
benchmark_cv = CrossValidationBenchmark(
    model_classes_and_params=model_classes_and_params, # iterate over this list of tuples
    datasets=dataset_list, # iterate over this list
    metrics=["MAE", "MSE"],
    test_percentage=10,
    num_folds=3,
    fold_overlap_pct=0,
)
results_summary, results_train, results_val = benchmark_cv.run()

  0%|          | 0/202 [00:00<?, ?it/s]

  0%|          | 0/205 [00:00<?, ?it/s]

  0%|          | 0/207 [00:00<?, ?it/s]

  0%|          | 0/202 [00:00<?, ?it/s]

  0%|          | 0/205 [00:00<?, ?it/s]

  0%|          | 0/207 [00:00<?, ?it/s]

In [14]:
results_summary

,data,model,params,train_MAE,train_MAE_std,train_MSE,train_MSE_std,val_MAE,val_MAE_std,val_MSE,val_MSE_std
0,air_passengers,NeuralProphet,{},11.995695,2.315776,268.377174,96.296339,31.766712,1.176802,1391.404582,39.272640
1,air_passengers,NeuralProphet,{'n_changepoints': 5},12.645453,2.328012,279.106695,97.374211,32.275642,1.657967,1480.973063,81.804540
2,air_passengers,NeuralProphet,"{'seasonality_mode': 'multiplicative', 'learni...",5.974563,0.550465,60.886582,10.902528,20.221582,7.132781,570.315328,289.548278


In [16]:
# remove temporary data folder.
!rm -r tmp-data

rm: cannot remove 'tmp-data': No such file or directory
